# CanICA

In [ ]:
# ToDo: Add References and credits to NiLearn documentation

In [ ]:
import glob
from nilearn.decomposition import CanICA
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show

In [ ]:
# Visualization of resting state networks
# Show networks using plotting utilities
from nilearn import plotting

In [ ]:
import pandas as pd
import os

In [ ]:
os.chdir('/home/WorkFlowTemp/functional_analysis_mar_2019/20190701')

In [ ]:
# CONN denoised data
denoised_data = glob.glob('./conn_3/results/preprocessing/niftiDATA_Subject*_Condition000.nii')

In [ ]:
denoised_data.sort()
denoised_data[0]

## Indipendent Component Analysis

In [ ]:
ica = CanICA(n_components=5,
                memory="nilearn_cache/alldiag5", memory_level=2,
                threshold='auto',
                n_init=1,
                verbose=1,
                mask_strategy='template',
                n_jobs = 1)
ica.fit(denoised_data)
canica_components_img = ica.components_img_
plotting.plot_prob_atlas(canica_components_img, 
                         view_type='filled_contours',
                         title='CanICA_alldiag_{}c_templatemask maps'.format(n_components),
                         output_file = './CanICA_alldiag_{}c_templatemask_prob_atlas.png'.format(n_components))
for i, cur_img in enumerate(iter_img(canica_components_img)):
    plotting.plot_stat_map(cur_img, title='CanICA_alldiag_{n_components}c_templatemask map_{component}'.format(n_components=n_components, component=i))
    cur_img.to_filename('CanICA_alldiag_{n_components}c_templatemask_ROI_{component}.nii.gz'.format(n_components=n_components, component=i))

## Split every ICA component in the individuals ROIs

In [ ]:
# Import Region Extractor algorithm from regions module
# threshold=0.5 indicates that we keep nominal of amount nonzero voxels across all
# maps, less the threshold means that more intense non-voxels will be survived.
from nilearn.regions import RegionExtractor

extractor = RegionExtractor(canica_components_img, threshold=0.5,
                            thresholding_strategy='ratio_n_voxels',
                            extractor='local_regions',
                            standardize=True, min_region_size=1350)
# Just call fit() to process for regions extraction
extractor.fit()
# Extracted regions are stored in regions_img_
regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img.shape[-1]

# Visualization of region extraction results
title = ('%d regions are extracted from %d components.'
         '\nEach separate color of region indicates extracted region'
         % (n_regions_extracted, 5))
plotting.plot_prob_atlas(regions_extracted_img, view_type='filled_contours',
                         title=title)

for i, cur_img in enumerate(iter_img(regions_extracted_img)):
    plotting.plot_stat_map(cur_img, title='Extractor_alldiag_5c_templatemask map_{component}'.format(component=i))
    cur_img.to_filename('Extractor_alldiag_5c_templatemask_ROI_{component}.nii.gz'.format(component=i))